In [ ]:
'''
參考網址
1. http://free-proxy.cz/zh/proxylist/country/US/https/uptime/all
2. https://www.us-proxy.org/
'''


import requests as req
from bs4 import BeautifulSoup as bs
import re, base64
from random import randint
from pprint import pprint

# 操作 browser 的 API
from selenium import webdriver

# 處理逾時例外的工具
from selenium.common.exceptions import TimeoutException

# 面對動態網頁，等待某個元素出現的工具，通常與 exptected_conditions 搭配
from selenium.webdriver.support.ui import WebDriverWait

# 搭配 WebDriverWait 使用，對元素狀態的一種期待條件，若條件發生，則等待結束，往下一行執行
from selenium.webdriver.support import expected_conditions as EC

# 期待元素出現要透過什麼方式指定，通常與 EC、WebDriverWait 一起使用
from selenium.webdriver.common.by import By

# 強制等待 (執行期間休息一下)
from time import sleep, time

In [ ]:
# 設定請求標頭
my_headers = {
    'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.61 Safari/537.36 Edg/94.0.992.31'
}

# 整理所有取得資料的變數
listData = []

# 欲抓取資料的網址
domainName = 'https://www.wine-searcher.com'
url = domainName + '/find/mouton+rothschild+pauillac+medoc+bordeaux+france'

# proxy 設定
my_proxies = {
    "http": "",
    "https": "",
}

# 放置 proxy 列表的變數，每次用完或確認無法使用，就會拋棄，用完了，會再重新請求
listProxyPool = []

# 若是用完 proxy pool 造成例外，則重新取得，若錯誤次數過多，則結束程式
limit = 10
count = 0

# 啟動瀏覽器工具的選項
my_options = webdriver.ChromeOptions()
# my_options.add_argument("--headless")                #不開啟實體瀏覽器背景執行
my_options.add_argument("--start-maximized")         #最大化視窗
my_options.add_argument("--incognito")               #開啟無痕模式
my_options.add_argument("--disable-popup-blocking") #禁用彈出攔截
my_options.add_argument("--disable-notifications")  #取消通知
my_options.add_argument("--lang=zh-TW")  #設定為正體中文

# 指定 chromedriver 檔案的路徑
driver_exec_path = './chromedriver.exe'

# 使用 Chrome 的 WebDriver
driver = webdriver.Chrome( 
    options = my_options, 
    executable_path = driver_exec_path
)

In [ ]:
'''
函式
'''
# 取得 free proxy 列表 (https://free-proxy-list.net/)
def getProxyList():
    global listProxyPool
    global my_proxies
    global driver
    
    # 若先前有 proxy 資料，則先清空
    listProxyPool = []
    
    # 計算開始時間
    time1 = time()
    
    try:
        # 取得 html
        driver.get('https://www.sslproxies.org/')
        
        # 等待互動元素出現
        WebDriverWait(driver, 30).until(
            EC.presence_of_element_located(
                (By.CSS_SELECTOR, 'ul.share-buttons.hidden-print > li > a[title="Get raw list"]')
            )
        )
        
        # 點擊 Get raw list 圖示
        driver.find_element(By.CSS_SELECTOR, 'ul.share-buttons.hidden-print > li > a[title="Get raw list"]').click()
        
        # 等待
        sleep(2)
        
        # 取得列表內文
        text = driver.find_element(By.CSS_SELECTOR, "div#raw div.modal-body textarea").get_attribute("value")
        
        # 過濾出所有 ip
        regexIP = r"\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}:\d{2,5}"
        
        # 取得全部 ip
        list_ip = re.findall(regexIP, text)
        
        # 目前有多少 ip 可進行測試
        print(f"測試 IP 數: {len(list_ip)}")
        
        # 檢查每個 ip 是否可用
        for index, ip in enumerate(list_ip):
            try:
                my_proxies['http'] = ip
                my_proxies['https'] = ip
                res = req.get('https://ip.seeip.org/jsonip', proxies = my_proxies, timeout = 2)
                if res.ok:
                    print(f"流水號: {index} => 【可用 IP: {res.json()}】")
                    
                    # 整理可用資料
                    listProxyPool.append(ip)
                else:
                    print(f"流水號: {index} => ...IP: {ip} 無法使用...")
            except:
                print(f"流水號: {index} => ...IP: {ip} 無法使用...")
                
        print(f"可用 IP 列表 ({len(listProxyPool)}):")
        pprint(listProxyPool)
        
        # 計算結束時間
        print("取得可用 IP 列表花費 %2.4f 秒" % (time()-time1))

        # 取得第一組 proxy
        ip = listProxyPool.pop(0)
        my_proxies['http'] = ip
        my_proxies['https'] = ip
    except TimeoutException:
        print("等待逾時...")
    
# 取得首頁列表資訊
def getMainData():
    global limit
    global count
    global listProxyPool
    global my_proxies
    
    try:
        print("=" * 15 + "開始測試請求 (次數 5)" + "=" * 15)
        
        for i in range(5):
            res = req.get(url, headers = my_headers, proxies = my_proxies, stream=True, timeout = 2)
            if res.ok:
                print(f"本機對外連線資訊: {res.raw._connection.sock.getsockname()}")

                # 簡單確認回傳結果，若包括 automation 字眼，代表可能被擋，要換 ip
                regex = r"automation"
                match = re.search(regex, res.text)
                if match != None:
                    # 取得第一組 proxy
                    ip = listProxyPool.pop(0)
                    my_proxies['http'] = ip
                    my_proxies['https'] = ip

                    #再執行一次函式，重新抓取
                    getMainData()
                    
                    break
                else:
                    print("***結果: 成功，請求 (request) 已得到回應 (response)***")
                    print(f"當前可用 IP: {my_proxies}")
                    sleep( randint(2,5) )
    except:
        if len(listProxyPool) > 0:
            # 取得第一組 proxy
            ip = listProxyPool.pop(0)
            my_proxies['http'] = ip
            my_proxies['https'] = ip
        else:
            # 超過指定錯誤次數，則結束函式執行
            if count < limit:
                count += 1
            else:
                print("***結果: 失敗，超過指定次數***")
                return False
            
            # 當前使用 proxy pool 網站，每 10 分鐘更新一次，所以可以強制等待到它更新
#             sleep(600)
            
            # 尚未超過錯誤次數，則繼續執行程式
            getProxyList()
        
        #再執行一次函式，重新抓取
        getMainData()

In [ ]:
'''
主程式
'''
if __name__ == "__main__":
    # testProxy()
    time1 = time()
    getProxyList()
    getMainData()
    print("花費 %2.4f 秒" % (time()-time1))
    print(f"限制重新取得 Proxy 列表次數: {limit}, 實際重新取得次數: {count}")
    count = 0